# Plots for supplementary materials of paper
<a class="anchor" id="top"></a>
<p align="right">

Variable is
+ $\eta_0$ - reference viscosity [$10^{14},10^{24}$]Pas - params_2

Fixed values are:
+ $\phi_{RCMF}$=0.3
+ $\beta=0.0225\,K^{-1}$
+ $\eta_l=10$ Pas
+ $\alpha_n=30$
+ $^{60}Fe/^{56}Fe = 10^{-8}$
+ $X_{S,0}=29.85$ wt%
+ r=300km

This notebook creates the plots to justify explanantions in the EPSL paper which are included in the supplementary material. 
    
### Plots
+ [Reference viscosity](#eta) 
    + [3 epochs of dynamo generation $\eta_0$=$10^{20}$Pas](#3eta)
+ [Critical melt fraction](#rcmf)
    + [3 epochs of dynamo generation $\phi_C$=0.26](#3rcmf)

# Set-up
This can be run once for a specific variable

## Import libraries 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mpl
import seaborn as sns

Colours for plotting (there are a lot as there are a lot of values of $\eta_0$)

In [ ]:
colors = ['black','darkblue','blue','royalblue','cornflowerblue','paleturquoise','salmon','maroon','saddlebrown','chocolate','orange','tan','olive','olivedrab','yellowgreen','darkolivegreen']

## Import data

In [ ]:
folder = 'Paper_run300km/'
savefolder ='EPSL_paper/'
subfolders = {'rcmf':1,'eta0':2,'beta':3,'etal':4,'Xs_0':5,'Fe0':6,'alpha_n':7,'r':8}
labels = {'rcmf':'$\\phi_{{C}}$','eta0':'$\\eta_0$','beta':'$\\beta$','etal':'$\\eta_l$ ','Xs_0':'$X_{{s,0}}$','Fe0':'$^{{60}}Fe/^{{56}}Fe$','alpha_n':'$\\alpha_n$','r':'radius'}
Myr = 365*24*3600*1e6 #number of s in Myr
save = True

Import the summary info

In [ ]:
var_results = pd.read_csv(f'../Results_combined/{folder}/all_sucess_info.csv',skiprows=[1])

In [ ]:
r = 300e3
dr = 500
rcr = 0.5

In [ ]:
n_cells = int(r/dr)
nccells = round((n_cells-3)*(rcr))+2 #number of cells needed to span core (inc. centre and CMB)
nmantle = n_cells - nccells +1 #number of cells needed to span mantle plus one extra for CMB

## $\eta_0$ Stagnant lid thickness <a class="anchor" id="eta"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Choose variable of interest

In [ ]:
var = 'eta0'
unit = 'Pas' #unit of variable
varlab = labels[var]
logvar = True
path = '../Results_combined/'+folder+f"params_{subfolders[var]}/"

Find run numbers

In [ ]:
var_data = pd.read_csv(path+'auto_params.csv',skiprows=[1])
minrun = min(var_data['run'])
maxrun = max(var_data['run'])
nrun = len(var_data)
data = var_results[(var_results['run']>=minrun)&(var_results['run']<=maxrun)].copy(deep=True)
data.reset_index(inplace=True,drop=True)

In [ ]:
plt.figure(tight_layout=True,figsize=[10,8])
plt.suptitle(f'Mantle top and bottom boundary layer thickness for changes in {varlab}')
plt.subplot(2,1,1)
for i in range(nrun):
    run = int(minrun+i)
    varval = var_data.loc[var_data['run']==run,var].values[0]
    rmantle = var_data.loc[var_data['run']==run,'r'].values[0]/2
    fcond_t = var_results.loc[var_results['run']==run,'fcond_t'].values[0]
    tsolid = var_results.loc[var_results['run']==run,'tsolid'].values[0]
    if var=='r':
        varval = varval/1e3
    #import data
    npzfile = np.load(f'{path}run_{run}.npz')
    t = npzfile['t']/Myr #time in Myr
    d0 = npzfile['d0']
    dl = npzfile['dl']
    if fcond_t > 0:
        plt.plot(t[t<=fcond_t],1-d0[t<=fcond_t]/rmantle,label=f'{varlab}={varval:.2g}{unit}',color=colors[i])
        plt.plot(t[t<=fcond_t],dl[t<=fcond_t]/rmantle,color=colors[i],linestyle='dashed')
        
    else:
        plt.plot(t,1-d0/rmantle,label=f'{varlab}={varval:.2g}{unit}',color=colors[i])
        plt.plot(t,dl/rmantle,color=colors[i],linestyle='dashed')
        plt.vlines(tsolid,(1-d0[-1]/rmantle)-0.05,(1-d0[-1]/rmantle)+0.05,linestyle='dotted',color='gray')
 
plt.xlabel('Time after CAI formation/ Ma')
plt.ylabel('Height in mantle/mantle thickness')
plt.legend()
plt.xscale('log')
plt.xlim([0.8,240])
plt.ylim([0,1])
if save == True:
    plt.savefig(f'../Plots/{savefolder}d0_{var}.png',dpi=450,bbox_inches='tight')

## 3 epochs of dynamo generation <a class="anchor" id="3eta"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=1,sharex='col',tight_layout=True,figsize=[10,4])

for i in range(5,nrun-3): #only plot values either isde of eta0 = 10^20
    run = int(minrun+i)
    varval = var_data.loc[var_data['run']==run,var].values[0]
    rmantle = var_data.loc[var_data['run']==run,'r'].values[0]/2
    fcond_t=var_results.loc[var_results['run']==run,'fcond_t'].values[0]
    #import data
    npzfile = np.load(f'{path}run_{run}.npz')
    t = npzfile['t']/Myr #time in Myr
    Rem = npzfile['Rem']
    ax.plot(t,Rem,label=f'{varlab}={varval:.2g}{unit}',color=colors[i])
 
        
ax.hlines(10,0,70,linestyle='dashed',color='gray')
ax.set_xlabel('Time after CAI formation/ Ma')
ax.set_ylabel('Magnetic Reynolds number')
ax.legend()
ax.set_xlim([1.2,70])
ax.set_ylim([0,35])
if save == True:
    plt.savefig(f'../Plots/{savefolder}Rem_{var}.png',dpi=450,bbox_inches='tight')

# $\phi_{RCMF}$ <a class="anchor" id="rcmf"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
var = 'rcmf'
unit = '' #unit of variable
varlab = labels[var]
logvar = False
path = '../Results_combined/'+folder+f"params_{subfolders[var]}/"

Find run numbers

In [ ]:
var_data = pd.read_csv(path+'auto_params.csv',skiprows=[1])
minrun = min(var_data['run'])
maxrun = max(var_data['run'])
nrun = len(var_data)
data = var_results[(var_results['run']>=minrun)&(var_results['run']<=maxrun)].copy(deep=True)
data.reset_index(inplace=True,drop=True)

## Stagnant lid thickness

In [ ]:
fig,ax = plt.subplots(nrows=2,ncols=1,sharex='col',tight_layout=True,figsize=[10,8])

for i in range(nrun):
    run = int(minrun+i)
    varval = var_data.loc[var_data['run']==run,var].values[0]
    rmantle = var_data.loc[var_data['run']==run,'r'].values[0]/2
    fcond_t=var_results.loc[var_results['run']==run,'fcond_t'].values[0]
    #import data
    npzfile = np.load(f'{path}run_{run}.npz')
    t = npzfile['t']/Myr #time in Myr
    d0 = npzfile['d0']
    dl = npzfile['dl']
    Tm_conv = npzfile['Tm_conv']
    if fcond_t > 0:
        ax[0].plot(t[(t<=fcond_t)&(d0<rmantle)],1-d0[(t<=fcond_t)&(d0<rmantle)]/rmantle,label=f'{varlab}={varval:.2g}',color=colors[i])
        if i==0:
            ax[1].plot(t[(t<=fcond_t)&(d0<rmantle)],dl[(t<=fcond_t)&(d0<rmantle)]/rmantle,linestyle='dashed',color=colors[i],label='$\delta_l$')
        else:
            ax[1].plot(t[(t<=fcond_t)&(d0<rmantle)],dl[(t<=fcond_t)&(d0<rmantle)]/rmantle,linestyle='dashed',color=colors[i])
    else:
        ax[0].plot(t,d0/rmantle,label=f'{var}={varval:.2g}',color=colors[i])
    if i==0:
        ax[0].vlines(t[(Tm_conv-1400)/(400)<varval][0],0,1,color=colors[i],linestyle='dotted',label='$T_m<T_{\phi_C}$')
        ax[1].vlines(t[(Tm_conv-1400)/(400)<varval][0],0,1,color=colors[i],linestyle='dotted',label='$T_m<T_{\phi_C}$')
    else:
        ax[0].vlines(t[(Tm_conv-1400)/(400)<varval][0],0,1,color=colors[i],linestyle='dotted')
        ax[1].vlines(t[(Tm_conv-1400)/(400)<varval][0],0,1,color=colors[i],linestyle='dotted')
ax[1].set_xlabel('Time after CAI formation/ Ma')
ax[0].set_ylabel('Base of $\delta_0$ in mantle/mantle thickness')
ax[1].set_ylabel('Top of $\delta_l$ in mantle/mantle thickness')
ax[0].legend(loc='center left')
ax[1].legend()
ax[1].set_yscale('log')
ax[0].set_xlim([1.2,20])
ax[0].set_ylim([0.6,1])
if save == True:
    plt.savefig(f'../Plots/{savefolder}d0_{var}.png',dpi=450,bbox_inches='tight')

## Magnetic Reynolds number - 3 epochs of dynamo generation <a class="anchor" id="3rcmf"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
plt.figure(tight_layout=True,figsize=[10,8])
plt.subplot(2,1,1)
for i in range(nrun):
    run = int(minrun+i)
    varval = var_data.loc[var_data['run']==run,var].values[0]
    rmantle = var_data.loc[var_data['run']==run,'r'].values[0]/2
    fcond_t = var_results.loc[var_results['run']==run,'fcond_t'].values[0]
    tsolid = var_results.loc[var_results['run']==run,'tsolid'].values[0]
    if var=='r':
        varval = varval/1e3
    #import data
    npzfile = np.load(f'{path}run_{run}.npz')
    t = npzfile['t']/Myr #time in Myr
    Rem = npzfile['Rem']
    plt.plot(t,Rem,color=colors[i],label=f'{varlab}={varval:.2g}')

plt.hlines(10,0,120,linestyle='dashed',color='gray')
plt.xlabel('Time after CAI formation/ Ma')
plt.ylabel('Magnetic Reynolds number')
plt.legend()
plt.xlim([0,80])
plt.ylim([0,50])
if save == True:
    plt.savefig(f'../Plots/{savefolder}Rem_{var}.png',dpi=450,bbox_inches='tight')